In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from googlenewsdecoder import gnewsdecoder
from urllib.parse import urlparse
from fake_useragent import UserAgent

In [7]:
import feedparser
from newspaper import Article
from urllib.parse import unquote

In [8]:
#Getting top news from google news 

def get_news(rss_url="https://news.google.com/rss?hl=en-US&gl=US&ceid=US:en"):
    response = requests.get(rss_url)
    soup = BeautifulSoup(response.content, 'xml')
    
    return [{
        'title': item.title.text,
        'source': item.source.text,
        'time': item.pubDate.text,
        'link': item.link.text
    } for item in soup.find_all('item')]

In [9]:
#Decoding the google news links to original source

def decode_google_url(google_url):
    """Decode Google News tracking URL to original source"""
    try:
        
        path_segments = urlparse(google_url).path.split('/')
        article_id = path_segments[-1] if path_segments else ""
        
        # Use the decoder package
        result = gnewsdecoder(
            google_url,
            interval=2,  # 2s delay between requests
            proxy=None  # Add proxy if getting blocked
        )
        
        if result['status']:
            return result['decoded_url']
        return google_url 
    
    except Exception as e:
        print(f"Decoding failed: {str(e)}")
        return google_url


In [10]:
news=get_news()
news

[{'title': 'Trump Administration News: State Dept. Ignores Order to Detail Return of Wrongly Deported Migrant - The New York Times',
  'source': 'The New York Times',
  'time': 'Sun, 13 Apr 2025 01:50:13 GMT',
  'link': 'https://news.google.com/rss/articles/CBMid0FVX3lxTFA0emk5M2ZlWmpmbHVqTkpPaUp2d0tTM1pwajdtUkR0WVExMHB1TDVDMHVyTEJ1TDlOQWItNEhpNzlzaXpwZmhSd0VGOEhWVTBQRnNYWFNweF9BSWVuNV9FQ1ZkSjB3TmI1azFKaFEyYTFmdnpMV0lv?oc=5'},
 {'title': 'Iran Seeks Sanctions Relief for Nuclear Limits in Talks With U.S. - WSJ',
  'source': 'WSJ',
  'time': 'Sun, 13 Apr 2025 01:49:00 GMT',
  'link': 'https://news.google.com/rss/articles/CBMimwFBVV95cUxNVDJyVTlsV3FISG1aQS15cTFhNkhSb0gxQzZUb3BuVk45LWRabVUxTkRzNHNRU2ZweENIVlBmR2hBcF9FaDNjYWlUR0NhR1FxUDJSRWhGNmtWOFc0RG9heVJsTU5KTEUwWVEwZUxqaE0zXzVMNTBKczgxd1c1dUdmZl9ldGZMUDE2Z1JRZlJOMXM5d2lwYVZnb01CSQ?oc=5'},
 {'title': 'Federal judge orders USDA to unfreeze funds to Maine - NPR',
  'source': 'NPR',
  'time': 'Sat, 12 Apr 2025 15:14:18 GMT',
  'link': 'http

In [11]:
for item in news:
    item['link'] = decode_google_url(item['link'])
    print(item['link'])

https://www.nytimes.com/live/2025/04/12/us/trump-administration-news
https://www.wsj.com/world/middle-east/u-s-iran-begin-high-stakes-nuclear-talks-in-oman-fc07cdce
https://www.npr.org/2025/04/12/nx-s1-5362976/maine-usda-unfreeze-janet-mills-trump
https://www.theguardian.com/us-news/2025/apr/12/trump-military-control-us-mexico-border
https://apnews.com/article/united-states-china-trade-tariff-business-volatility-223e4cd53165da6fbd64aff5652a1aff
https://www.washingtonpost.com/immigration/2025/04/12/one-million-deportations-goal/
https://abcnews.go.com/US/tourist-helicopter-crashed-hudson-flight-recorder-ntsb/story?id=120755978
https://timesofindia.indiatimes.com/world/us/what-will-happen-to-tuft-student-rumeysa-ozturk-as-judge-rules-for-deporting-columbias-mahmoud-khalil/articleshow/120240798.cms
https://www.nytimes.com/2025/04/12/us/idaho-abortion-medical-ban-exceptions-ruling.html
https://www.cnn.com/2025/04/12/us/autistic-teen-idaho-police-shooting-dies/index.html
https://www.nytimes

In [12]:
news

[{'title': 'Trump Administration News: State Dept. Ignores Order to Detail Return of Wrongly Deported Migrant - The New York Times',
  'source': 'The New York Times',
  'time': 'Sun, 13 Apr 2025 01:50:13 GMT',
  'link': 'https://www.nytimes.com/live/2025/04/12/us/trump-administration-news'},
 {'title': 'Iran Seeks Sanctions Relief for Nuclear Limits in Talks With U.S. - WSJ',
  'source': 'WSJ',
  'time': 'Sun, 13 Apr 2025 01:49:00 GMT',
  'link': 'https://www.wsj.com/world/middle-east/u-s-iran-begin-high-stakes-nuclear-talks-in-oman-fc07cdce'},
 {'title': 'Federal judge orders USDA to unfreeze funds to Maine - NPR',
  'source': 'NPR',
  'time': 'Sat, 12 Apr 2025 15:14:18 GMT',
  'link': 'https://www.npr.org/2025/04/12/nx-s1-5362976/maine-usda-unfreeze-janet-mills-trump'},
 {'title': 'Trump authorizes US military to take control of land at US-Mexico border - The Guardian',
  'source': 'The Guardian',
  'time': 'Sat, 12 Apr 2025 21:06:00 GMT',
  'link': 'https://www.theguardian.com/us-ne

In [13]:
import requests
from newspaper import Article
from fake_useragent import UserAgent
from requests.adapters import HTTPAdapter
from urllib3.util import Retry
import time

# Configure session with retry logic
session = requests.Session()
retries = Retry(
    total=3,
    backoff_factor=0.5,
    status_forcelist=[500, 502, 503, 504],
    allowed_methods=["GET"]
)
session.mount('https://', HTTPAdapter(max_retries=retries))

def get_full_article_content(article_url):
    """Enhanced article downloader with timeout handling"""
    headers = {
        'User-Agent': UserAgent().random,
        'Accept-Language': 'en-US,en;q=0.9',
        'Referer': 'https://news.google.com/',
        'DNT': '1'
    }
    
    try:
        response = session.get(article_url, headers=headers, timeout=(3.05, 10))
        response.raise_for_status()

        article = Article(article_url)
        article.download(input_html=response.text)
        article.parse()
        
        return {
            'title': article.title,
            'publish_date': article.publish_date,
            'text': article.text,
            'top_image': article.top_image,
            'source_url': article_url,
        }

    except requests.exceptions.Timeout:
        print(f"⏰ Timeout skipped: {article_url}")
        return None
    except Exception as e:
        print(f"⚠️ Error processing {article_url}: {str(e)}")
        return None

def process_news_items(news_items, delay=1):
    results = []
    for idx, item in enumerate(news_items, 1):
        print(f"Processing article {idx}/{len(news_items)}...")
        article = get_full_article_content(item['link'])
        if article:
            results.append(article)
        time.sleep(delay)
    return results


newslist = process_news_items(news)
newslist

Processing article 1/38...
Processing article 2/38...
Processing article 3/38...
Processing article 4/38...
Processing article 5/38...
Processing article 6/38...
Processing article 7/38...
Processing article 8/38...
Processing article 9/38...
Processing article 10/38...
Processing article 11/38...
Processing article 12/38...
Processing article 13/38...
⚠️ Error processing https://www.timesofisrael.com/hamas-releases-propaganda-video-showing-us-israeli-hostage-edan-alexander/: 403 Client Error: Forbidden for url: https://www.timesofisrael.com/hamas-releases-propaganda-video-showing-us-israeli-hostage-edan-alexander/
Processing article 14/38...
Processing article 15/38...
Processing article 16/38...
Processing article 17/38...
Processing article 18/38...
Processing article 19/38...
Processing article 20/38...
Processing article 21/38...
Processing article 22/38...
Processing article 23/38...
Processing article 24/38...
Processing article 25/38...
Processing article 26/38...
Processing ar

[{'title': 'Trump Administration News: State Dept. Ignores Order to Detail Return of Wrongly Deported Migrant',
  'publish_date': datetime.datetime(2025, 4, 12, 0, 0),
  'text': "The threat of higher iPhone prices caused some Americans to rush to Apple stores to buy new phones.\n\nAfter more than a week of ratcheting up tariffs on products imported from China, the Trump administration issued a rule late Friday that spared smartphones, computers, semiconductors and other electronics from some of the fees, in a significant break for tech companies like Apple and Dell and the prices of iPhones and other consumer electronics.\n\nA message posted late Friday by U.S. Customs and Border Protection included a long list of products that would not face the reciprocal tariffs President Trump imposed in recent days on Chinese goods as part of a worsening trade war. The exclusions would also apply to modems, routers, flash drives and other technology goods, which are largely not made in the United 

In [14]:
df = pd.DataFrame(newslist)

In [15]:
df

,title,publish_date,text,top_image,source_url
0,Trump Administration News: State Dept. Ignores...,2025-04-12 00:00:00,The threat of higher iPhone prices caused some...,https://static01.nyt.com/images/2025/04/12/mul...,https://www.nytimes.com/live/2025/04/12/us/tru...
1,Iran Seeks Sanctions Relief for Nuclear Limits...,None,The indirect talks were held at a secure compo...,https://images.wsj.net/im-13976640/social,https://www.wsj.com/world/middle-east/u-s-iran...
2,Federal judge orders USDA to unfreeze funds to...,2025-04-12 00:00:00,Federal judge orders USDA to unfreeze funds to...,https://npr.brightspotcdn.com/dims3/default/st...,https://www.npr.org/2025/04/12/nx-s1-5362976/m...
3,Trump authorizes US military to take control o...,2025-04-12 00:00:00,Donald Trump has authorized the military to ta...,https://i.guim.co.uk/img/media/ef31ffc06a30850...,https://www.theguardian.com/us-news/2025/apr/1...
4,As tariffs put trade between China and the US ...,2025-04-13 04:01:58,"When the first two rounds of 10% tariffs hit, ...",https://dims.apnews.com/dims4/default/18a6ef3/...,https://apnews.com/article/united-states-china...
5,‘One million.’ The private goal driving Trump’...,2025-04-12 00:00:00,As the Trump administration aggressively pushe...,https://www.washingtonpost.com/wp-apps/imrs.ph...,https://www.washingtonpost.com/immigration/202...
6,Tourist helicopter that crashed into the Hudso...,None,The helicopter was on its eighth flight of the...,https://i.abcnewsfe.com/a/7d07591f-06b7-4c6d-a...,https://abcnews.go.com/US/tourist-helicopter-c...
7,What will happen to Tuft student Rumeysa Oztur...,None,The verdict of the Mahmoud Khalil case was cru...,https://static.toiimg.com/thumb/msid-120241066...,https://timesofindia.indiatimes.com/world/us/w...
8,Idaho Court Expands Abortion Ban Medical Excep...,2025-04-12 00:00:00,A state judge in Idaho appeared to slightly br...,https://static01.nyt.com/images/2025/04/12/mul...,https://www.nytimes.com/2025/04/12/us/idaho-ab...
9,Autistic teen shot by Idaho police dies after ...,2025-04-12 00:00:00,"Boise, Idaho AP —\n\nAn autistic, nonverbal te...",https://media.cnn.com/api/v1/images/stellar/pr...,https://www.cnn.com/2025/04/12/us/autistic-tee...


# Time to do ai analysis

In [17]:

import numpy as np # linear algebra
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import nltk
from nltk.corpus import stopwords
import pickle

# 1. Create and fit a temporary vectorizer with sample vocabulary
with open("vectorizer.pkl", "rb") as f:
    vectorizer = pickle.load(f)
# 2. Model setup with original architecture
model = Sequential([
    Dense(100, activation='relu', input_dim=1000),
    Dense(50, activation='relu'),
    Dense(25, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')
])

# 3. Load weights manually
weights = np.load(r'C:\Users\Ashwin\Desktop\MAD_LAB_PROJECT\NewsScraper\model_weights.npy', allow_pickle=True)
for layer, (weights, bias) in zip(model.layers, weights.reshape(-1, 2)):
    layer.set_weights([weights, bias])
    
nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('english'))


In [18]:
print("Vectorizer feature count:", len(vectorizer.get_feature_names_out()))
print("Model input weights shape:", model.layers[0].get_weights()[0].shape)


Vectorizer feature count: 1000
Model input weights shape: (1000, 100)


In [ ]:

def preprocess(text):
    """Basic text cleaning"""
    return ' '.join([
        word.lower() 
        for word in text.split() 
        if word.lower() not in stop_words and word.isalpha()
    ])
def analyze_article(article, top_n=5):
    """
    Predict whether an article is Fake or Real, with confidence and key words.

    Args:
    article (str): The text of the article to analyze.
    top_n (int): Number of top influential words to return.

    Returns:
    dict: Containing prediction, confidence, and top influential words.
    """
    # Preprocess the article
    processed = preprocess(article)

    # Transform the article into a BOW representation
    bow_vector = vectorizer.transform([processed]).toarray()
    print("BOW vector shape:", bow_vector.shape)
    # Pad the BOW vector to match the model's input shape (1023669 features)
    padded_bow = np.zeros((100, 1000))  # Create a zero array with required shape
    for i in range(padded_bow.shape[0]):
        padded_bow[i, :bow_vector.shape[1]] = bow_vector  # Copy existing features into padded array

    # Get prediction confidence
    confidence = model.predict(padded_bow, verbose=0)[0][0]
    prediction = 'Fake' if confidence > 0.5 else 'Real'

    # Get feature importance using first layer weights
    input_weights = model.layers[0].get_weights()[0]  # First layer weights
    feature_importance = input_weights.dot(padded_bow)  # Multiply BOW by weights

    # Extract only words present in the article (non-zero BOW entries)
    feature_names = vectorizer.get_feature_names_out()
    non_zero_indices = np.nonzero(bow_vector[0])[0]  # Indices of non-zero features
    print(input_weights.shape,feature_importance.shape,feature_names.shape)
    keywords_with_scores = [(feature_names[i], feature_importance[0, i]) for i in non_zero_indices]

    # Sort by absolute impact score and select top N keywords
    sorted_keywords = sorted(keywords_with_scores, key=lambda x: abs(x[1]), reverse=True)[:top_n]
    return {
        'prediction': prediction,
        'confidence': float(confidence if prediction == 'Fake' else 1 - confidence),
        'keywords': sorted_keywords
    }

In [66]:
for i in newslist:
    topimage=i['top_image']
    asource=i['source_url']
    atitle=i['title']
    text=i['text']
    summary=text_summarizer(text)
    analysis = analyze_article(text)
    confidence_rating = analysis['confidence']
    confidence_rating=round(confidence_rating * 100, 1)
    keywords = analysis['keywords']
    words=[]
    
    for w,s in keywords:
        words.append(w)
    print("Title: ", atitle)
    print("Confidence: ", confidence_rating)
    print("Keywords: ", keywords)

BOW vector shape: (1, 1000)
(1000, 100) (1000, 1000) (1000,)
Title:  Trump Administration News: State Dept. Ignores Order to Detail Return of Wrongly Deported Migrant
Confidence:  0.1
Keywords:  [('trump', np.float64(6.221980425179936)), ('said', np.float64(5.80718173016794)), ('would', np.float64(5.3923830351559445)), ('administration', np.float64(4.977584340143949)), ('states', np.float64(4.147986950119957))]
BOW vector shape: (1, 1000)
(1000, 100) (1000, 1000) (1000,)
Title:  Iran Seeks Sanctions Relief for Nuclear Limits in Talks With U.S.
Confidence:  26.1
Keywords:  [('next', np.float64(1.2443960850359872)), ('foreign', np.float64(0.8295973900239915)), ('nuclear', np.float64(0.8295973900239915)), ('said', np.float64(0.8295973900239915)), ('held', np.float64(0.41479869501199573))]
BOW vector shape: (1, 1000)
(1000, 100) (1000, 1000) (1000,)
Title:  Federal judge orders USDA to unfreeze funds to Maine
Confidence:  47.5
Keywords:  [('trump', np.float64(2.4887921700719744)), ('order'

# Summariser

In [20]:
import nltk 
nltk.download('stopwords') 
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ashwin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ashwin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Ashwin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [36]:
import random
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.probability import FreqDist

def text_summarizer(text, num_sentences=3):
    # Text into sentences
    sentences = sent_tokenize(text)

    # Text into words
    words = word_tokenize(text.lower())

    # Removing stop words
    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in words if word.casefold() not in stop_words]

    # Calculate word frequencies
    fdist = FreqDist(filtered_words)

    # Assign scores to sentences based on word frequencies
    sentence_scores = [sum(fdist[word] for word in word_tokenize(sentence.lower()) if word in fdist)
                       for sentence in sentences]

    # Create a list of tuples containing sentence index and score
    sentence_scores = list(enumerate(sentence_scores))

    # Sort sentences by scores in descending order
    sorted_sentences = sorted(sentence_scores, key=lambda x: x[1], reverse=True)

    # Randomly select the top `num_sentences` sentences for the summary
    random_sentences = random.sample(sorted_sentences, num_sentences)

    # Sort the randomly selected sentences based on their original order in the text
    summary_sentences = sorted(random_sentences, key=lambda x: x[0])

    # Create the summary
    summary = ' '.join([sentences[i] for i, _ in summary_sentences])

    return summary

# Firebase uploader

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore

# Initialize Firebase
cred = credentials.Certificate(r"C:\Users\Ashwin\Desktop\MAD_LAB_PROJECT\NewsScraper\truthlens-68bed-firebase-adminsdk-fbsvc-4c3626b3ec.json")  #Get your own firebase credentials and put here
firebase_admin.initialize_app(cred)

In [ ]:


# Get Firestore database
db = firestore.client()

def upload_article(title, article, confidence, image, source, tags):
    """
    Uploads an article to the Firestore collection 'articles'.
    
    Args:
        title (str): Title of the article.
        article (str): Full text of the article.
        confidence (float): Confidence score of the article.
        image (str): URL of the article image.
        source (str): Source URL of the article.
        tags (list): List of tags related to the article.

    Returns:
        str: Document ID of the uploaded article.
    """
    data = {
        "title": title,
        "article": article,
        "confidence": confidence,
        "image": image,
        "source": source,
        "tags": tags
    }
    
    doc_ref = db.collection("news").add(data)
    return doc_ref[1].id  # Returns the auto-generated document ID


# Final output

In [26]:
for i in newslist:
    topimage=i['top_image']
    asource=i['source_url']
    atitle=i['title']
    text=i['text']
    summary=text_summarizer(text)
    analysis = analyze_article(text)
    confidence_rating = analysis['confidence']
    confidence_rating=round(confidence_rating * 100, 1)
    keywords = analysis['keywords']
    words=[]
    
    for w,s in keywords:
        words.append(w)
    print("Title: ", atitle)
    print("Confidence: ", confidence_rating)
    print("Keywords: ", keywords)

IndexError: index 1 is out of bounds for axis 0 with size 1